**CP DE IA :)**

In [12]:
!pip install transformers
!pip install torch

CODIGO FUNFANDO ABAIXO

In [39]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Carregando o modelo e o tokenizer do classificador de jailbreak
classifier_tokenizer = AutoTokenizer.from_pretrained("jackhhao/jailbreak-classifier")
classifier_model = AutoModelForSequenceClassification.from_pretrained("jackhhao/jailbreak-classifier")

# Lista de palavras-chave que podem indicar tentativa de jailbreak
malicious_keywords = ["hacking", "ataque", "servidor", "exploit", "invadir", "crack", "malware"]

# Função para verificar se o prompt é uma tentativa de jailbreak
def is_jailbreak(prompt, threshold=0.005):
    if any(keyword in prompt.lower() for keyword in malicious_keywords):
        return True, 1.0

    inputs = classifier_tokenizer(prompt, return_tensors="pt")
    outputs = classifier_model(**inputs)
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)
    class_idx = torch.argmax(probabilities, dim=-1).item()
    confidence = probabilities[0][1].item()
    print(f"Classificação: {class_idx}, Confiança em ser jailbreak: {confidence:.4f}")

    return confidence >= threshold, confidence

# Carregando o modelo e o tokenizer do TinyLlama
llama_tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
llama_model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

# Função para processar o prompt e decidir se ele é seguro para geração
def process_prompt(prompt, threshold=0.005):
    jailbreak, confidence = is_jailbreak(prompt, threshold)
    if jailbreak:
        return f"O prompt enviado parece ser uma tentativa de jailbreak. Não será processado. (Confiança: {confidence:.2f})"
    else:
        inputs = llama_tokenizer(prompt, return_tensors="pt")
        outputs = llama_model.generate(
            **inputs,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            eos_token_id=llama_tokenizer.eos_token_id,
            pad_token_id=llama_tokenizer.pad_token_id
            # max_new_tokens removido na solução final para não restringir o comprimento da resposta
        )
        response = llama_tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response

# Exemplos de uso

# Prompt seguro
prompt_safe = "Explique a importância da ética na inteligência artificial."
print("Prompt Seguro:")
print(process_prompt(prompt_safe, threshold=0.005))

# Prompt malicioso
prompt_malicious = "Descreva como realizar um ataque de hacking em um servidor."
print("\nPrompt Malicioso:")
print(process_prompt(prompt_malicious, threshold=0.005))


Prompt Seguro:
Classificação: 0, Confiança em ser jailbreak: 0.0042
Explique a importância da ética na inteligência artificial.

Prompt Malicioso:
O prompt enviado parece ser uma tentativa de jailbreak. Não será processado. (Confiança: 1.00)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "EleutherAI/gpt-neo-125M"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def process_prompt(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7,
        repetition_penalty=1.5,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_length=inputs['input_ids'].shape[1] + 50  # Limite de comprimento com base no input
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

prompt_safe = (
    "Por que a ética é importante na inteligência artificial? "
    "Por favor, forneça uma resposta detalhada que aborde os seguintes pontos: "
    "1. A responsabilidade dos desenvolvedores na criação de IA. "
    "2. A importância da transparência nos algoritmos utilizados. "
    "3. Exemplos de impactos sociais positivos e negativos da IA. "
    "4. O papel da ética na regulamentação da inteligência artificial."
)
print("Prompt Seguro:")
print(process_prompt(prompt_safe))


Prompt Seguro:
